In [1]:
#Import libraries
import os
import geopandas as gp
import matplotlib.pyplot as plt
import json
from contextlib import redirect_stdout
import sys
if r'C:\Users\attic\HLS Kelp Detection\tools' not in sys.path:
    sys.path.append(r'C:\Users\attic\HLS Kelp Detection\tools')
import earth_access_tools as eat
import csv
import earthaccess
import rasterio
import numpy as np
import shutil 

In [2]:
#Login
earthaccess.login(persist=True)

In [3]:
#Set time and location
geojson_path = r'C:\Users\attic\HLS Kelp Detection\maps\Isla_Vista_Kelp.geojson'
field = gp.read_file(geojson_path)
bbox = tuple(list(field.total_bounds))
bbox #Display coordinate bounds
with open(geojson_path, 'r') as f:
    data = json.load(f)
# Extract the name
location = (data['name']).replace('.kmz', '').replace(' ', '_')
temporal = ("2018-07-01T00:00:00", "2020-01-01T00:00:00") #

#DEM map doesn't work great as is. For processing, I recommend downloading one from: https://www.ncei.noaa.gov/maps/bathymetry/ and inserting it into the /imagery folder
create_dem = False # set true if u want a digital elevation map 
dem_name = 'dem.tif'

num_to_download = 500 #set this value to the number of frames you want
load_num = 500 #sets number of granules to load, this should generally be >> than num_download; -1 loads all 
specific_tile = True #set true if you only want specific tile 
download_to_path = r'C:\Users\attic\HLS Kelp Detection\imagery'
tile = '11SKU' # 

threads = os.cpu_count() / 2 

In [5]:
#Search for satellite data from  Landsat 30m and Sentinel 30m
results = earthaccess.search_data(
    short_name=['HLSL30','HLSS30'],
    granule_id = tile,
    temporal=temporal,
     cloud_cover=0, #Determine cloud cover
    count=load_num
)
if create_dem:
    dem_results = earthaccess.search_data(
        short_name="ASTGTM",
        bounding_box=bbox)
    print(dem_results[0])

#print(results[0])


ValueError: Unknown key granule_id

In [8]:

folder_path = os.path.join(os.path.join(download_to_path,location))
## ======= create location folder path ======= ##
if not os.path.isdir(folder_path):
    os.mkdir(folder_path)

## ======= Create/write DEM, if requested ======= ##
if create_dem:
    if not os.path.isfile(os.path.join(folder_path, dem_name)):
            dem_path = earthaccess.download(dem_results[0], local_path=folder_path)
            os.rename(dem_path[0], os.path.join(folder_path,'dem.tif'))
            os.rename(dem_path[1], os.path.join(folder_path, 'num.tif'))

downloaded = 0

for i, granule in enumerate(results):
    metadata = extract_metadata(granule)
    ## ======= Parse metadata ======= ##

    name = metadata['native-id']

    #For some reason, attributes are parsed into a list in the HLS metadata. This reformats it into a dictionary.
    #print(attributes['MGRS_TILE_ID'])

    if(int(metadata['CLOUD_COVERAGE']) > 50): #Reject granules with large cloud cover, for now
        continue
    time = metadata['SENSING_TIME']
    tile_folder = metadata['MGRS_TILE_ID']
    if specific_tile and not tile_folder == tile:
        continue
    ## ======= Create file directory, if needed  ======= ##
    tile_path = os.path.join(folder_path,tile_folder)
    if not os.path.isdir(tile_path):
         os.mkdir(tile_path)
    folder_name = (f'{name}')
    file_path = os.path.join(tile_path,folder_name)
    if not os.path.isdir(file_path):
        os.mkdir(file_path) #Make folder for granule 

    # sorted data links 
    all_links = granule.data_links()
    links = get_band_links(all_links, name)

    ## ======= download granule ======= ##
    with open(os.devnull, 'w') as f, redirect_stdout(f): #The print out of this is kind of annoying, this redirects *most* of it 
        downloadPath = earthaccess.download(links, local_path=file_path, threads=2)
    downloaded = downloaded + 1
    print(f'{name}')
    if downloaded > num_to_download:
         break

    ## ======= write metadata csv ======= ##
    save_metadata_csv(metadata,file_path,folder_name)


QUEUEING TASKS | :   0%|          | 0/7 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/7 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/7 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/7 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/7 [00:00<?, ?it/s]

...

QUEUEING TASKS | :   0%|          | 0/7 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/7 [00:00<?, ?it/s]

HLS.S30.T11SKU.2019362T184759.v2.0


COLLECTING RESULTS | :   0%|          | 0/7 [00:00<?, ?it/s]

HLS.L30.T11SKU.2019365T183420.v2.0


In [ ]:
#Alternatively, this recompiles the downloaded data into a single tif. 
#The previous method is probably better. Leaving this just in case.

# temp_path = r'C:\Users\attic\HLS Kelp Detection\temp'
# folder_path = os.path.join(os.path.join(download_to_path,location))
# ## ======= create location folder path ======= ##

# downloaded = 0

# for i, granule in enumerate(results):
#     if os.path.isdir(temp_path):
#        shutil.rmtree(temp_path)
#     os.mkdir(temp_path)
#     metadata = extract_metadata(granule)
#     ## ======= Parse metadata ======= ##

#     name = metadata['native-id']

#     #For some reason, attributes are parsed into a list in the HLS metadata. This reformats it into a dictionary.
#     #print(attributes['MGRS_TILE_ID'])

#     if(int(metadata['CLOUD_COVERAGE']) > 50): #Reject granules with large cloud cover, for now
#         continue
#     time = metadata['SENSING_TIME']
#     tile_folder = metadata['MGRS_TILE_ID']
#     if specific_tile and not tile_folder == tile:
#         continue
#     ## ======= Create file directory, if needed  ======= ##
#     tile_path = os.path.join(folder_path,tile_folder)
#     if not os.path.isdir(tile_path):
#          os.mkdir(tile_path)
#     file_path = os.path.join(tile_path,f'{name}.tif')
#     if os.path.isfile(file_path):
#         continue
#     # if not os.path.isdir(file_path):
#     #     os.mkdir(file_path) #Make folder for granule 

#     # sorted data links 
#     all_links = granule.data_links()
#     links = get_band_links(all_links, name)

#     ## ======= download granule ======= ##
#     #with open(os.devnull, 'w') as f, redirect_stdout(f): #The print out of this is kind of annoying, this redirects *most* of it 
#     downloadPath = earthaccess.download(links, local_path=temp_path)
#     bands = []
#     transform = None

#     for file in get_sorted_files(folder_path=temp_path,granule_name=name):
#         with rasterio.open(os.path.join(temp_path,file), 'r') as src:
#             bands.append(src.read())
#             if transform is None:
#                 transform = src.transform
#                 crs = src.crs

#     bands = np.vstack(bands)
#     print(bands.shape)
#     num_bands, height, width = bands.shape
#     data_type = rasterio.int16
#     profile = {
#         'driver': 'GTiff',
#         'width': width,
#         'height': height,
#         'count': num_bands,  # one band  B02, B03, B04, and B05, classified, mesma (Blue, Green, Red, and NIR).
#         'dtype': data_type,  # assuming binary mask, adjust dtype if needed
#         'crs': crs,
#         'transform': transform,
#         'nodata': 0,  # assuming no data is 0
#         'tags': metadata
#     }

#     with rasterio.open(file_path, 'w', **profile) as dst: #output file is a stack of 6 HLS bands (in chronological order) + Fmask as image 7
#         for i,band in enumerate(bands):
#             dst.write(band.astype(data_type), i + 1)

    
#     downloaded = downloaded + 1
#     shutil.rmtree(temp_path)
#     print(f'{name}')
#     if downloaded > num_to_download:
#          break
